1.1 **Initialization and Configuration**

* In this section, necessary libraries are imported, and the Semantic Kernel is configured with Azure OpenAI credentials loaded from a configuration file.

In [1]:
#!import config/Settings.cs

In [2]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta8

In [3]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [4]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.WithAzureOpenAIChatCompletionService(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.WithAzureOpenAIChatCompletionService(model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

1.2 **Semantic Function for Short Intent Extraction**
* This section focuses on setting up the Semantic Function for extracting a concise user intent using a specific prompt.

In [5]:
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.AI;

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt4ServiceId
};

In [6]:
var input = "I want to find top-10 books about world history";

In [7]:
// Chat with the Bot
// Create a simple prompt for the semantic function

string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 25 words or less: ";

var getShortIntentFunction  = kernel.CreateSemanticFunction(skPrompt, "fn", "pn", requestSettings: aiRequestSettings);

var intentResult = await kernel.RunAsync(input, getShortIntentFunction);

Console.WriteLine(intentResult);

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Service request failed.
Status: 401 (Unauthorized)

Headers:
Date: Sat, 09 Dec 2023 16:27:29 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 15

 ---> Azure.RequestFailedException: Service request failed.
Status: 401 (Unauthorized)

Headers:
Date: Sat, 09 Dec 2023 16:27:29 GMT
Connection: keep-alive
Strict-Transport-Security: REDACTED
Content-Length: 15

   at Azure.Core.HttpPipelineExtensions.ProcessMessageAsync(HttpPipeline pipeline, HttpMessage message, RequestContext requestContext, CancellationToken cancellationToken)
   at Azure.AI.OpenAI.OpenAIClient.GetChatCompletionsAsync(String deploymentOrModelName, ChatCompletionsOptions chatCompletionsOptions, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.ClientBase.RunRequestAsync[T](Func`1 request)
   --- End of inner exception stack trace ---
   at Microsoft.Semantic

1.3 **Semantic Function for Conversational Chat**

* This section defines the Semantic Function for engaging in a more extended conversation with the chatbot, initializing the context, and executing the function.

In [8]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

In [9]:
// Chat with the Bot

aiRequestSettings = new OpenAIRequestSettings
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5,
    ServiceId = gpt35TurboServiceId
};

var chatFunction = kernel.CreateSemanticFunction(skPrompt, aiRequestSettings);
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

var userInput = "Hi, I'm looking for suggestions of best sellers about world history";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

Error: (11,60): error CS1503: Argument 3: cannot convert from 'Microsoft.SemanticKernel.AI.AIRequestSettings' to 'string?'

In [10]:
// Update the history with the output and set this as the new input value for the next request

history += $"\nUser: {userInput}\nSuggestions: {bot_answer.GetValue<string>()}\n";
context.Variables.Update(history);

Console.WriteLine(context);

Error: (3,1): error CS0103: The name 'history' does not exist in the current context
(3,23): error CS0103: The name 'userInput' does not exist in the current context
(3,49): error CS0103: The name 'bot_answer' does not exist in the current context
(4,1): error CS0103: The name 'context' does not exist in the current context
(4,26): error CS0103: The name 'history' does not exist in the current context
(6,19): error CS0103: The name 'context' does not exist in the current context

1.4 **Interactive Chat Function**

* In this final section, an interactive chat function is defined, and several interactions with the chatbot are simulated. The complete chat history and the final user input are then printed. *

In [11]:
// Chatting more)

Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nSuggestions: {answer.GetValue<string>()}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

Error: (5,5): error CS0103: The name 'context' does not exist in the current context
(8,24): error CS0103: The name 'chatFunction' does not exist in the current context
(8,49): error CS0103: The name 'context' does not exist in the current context
(11,5): error CS0103: The name 'history' does not exist in the current context
(12,5): error CS0103: The name 'context' does not exist in the current context
(12,36): error CS0103: The name 'history' does not exist in the current context
(15,23): error CS0103: The name 'context' does not exist in the current context

In [12]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

Error: (1,7): error CS0103: The name 'Chat' does not exist in the current context

In [13]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

Error: (1,7): error CS0103: The name 'Chat' does not exist in the current context

In [14]:
await Chat("Which topic from the ones you listed do you think the most popular?");

Error: (1,7): error CS0103: The name 'Chat' does not exist in the current context

In [15]:
Console.WriteLine(context.Variables["history"]);

Error: (1,19): error CS0103: The name 'context' does not exist in the current context

In [16]:
Console.WriteLine(context.Variables["userInput"]);

Error: (1,19): error CS0103: The name 'context' does not exist in the current context

1.5 Switching between models.